# Applying ML to Predict NYC Real Estate Value & Investment Opportunity

*This notebook scrapes streeteasy.com for data on listings for sale in the Five Boroughs and applies ML to evaluate my predictive model*

In [1]:
from selenium import webdriver
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
import random
# from retrying import retry
import tenacity
# import json
# import os
# from collections import OrderedDict
# from urlparse import urlparse

In [2]:
boroughs = ['manhattan','brooklyn','queens','bronx','staten-island']

In [3]:
def url_def(lst):
    lst_of_urls = []
    for item in lst:
        lst_of_urls.append('http://streeteasy.com/for-sale/'+str(item)+'/status:listed?refined_search=true')
    return lst_of_urls

In [4]:
urls = url_def(boroughs)
urls

['http://streeteasy.com/for-sale/manhattan/status:listed?refined_search=true',
 'http://streeteasy.com/for-sale/brooklyn/status:listed?refined_search=true',
 'http://streeteasy.com/for-sale/queens/status:listed?refined_search=true',
 'http://streeteasy.com/for-sale/bronx/status:listed?refined_search=true',
 'http://streeteasy.com/for-sale/staten-island/status:listed?refined_search=true']

### Feature Selection

What would be the most useful features to collect for this project?

- **Type of house, Location of house, Neighborhood, Number of rooms, number of baths**, availability of amenities in building (laundry, doorman, super)?, proximity to transit, proximity to waterfront, (with the Price of house as target).

Features in bold are available on streeteasy...

Q. ***Can any useful features be engineered from those available or retrieved from an alternate source?***

### Scraped features

The features available from Streeteasy.com are:

 - House type, Geo-location, House address, No. of beds, No. of baths, Square area of house, Neighborhood, Price
 
*What features can be derived from these? What additional insight will these derived features provide?*
*Can more useful features be retrieved from other sources to complement Streeteasy?*

In [5]:
listing_type = []
lat = []
lng = []
address = []
no_of_beds = []
no_of_baths = []
sq_area = []
n_hood = []
borough = []
price = []

### *Thoughts*

Do the project requirements dictate the statistical method/algorithm used? Will these, in turn, determine whether categorical or continuous variables are required?

- *Linear Regression*
- *Logistic Regression*
- *Random forest*

**N.B. This requirement directly dictates the page_scrape function below.**

**I'm going with numerical variables where possible...**

In [6]:
def page_scrape(page):
    count = 0
    # On slow connections...
    # listings = WebDriverWait(page, 30).until(EC.visibility_of_element_located((By.ID, 'results-details')))
    listings = page.find_element_by_id('result-details').find_element_by_tag_name('ul').find_elements_by_tag_name('li')
    # collect data here by iterating through each listing and appending to our lists
    for l in listings[:14]:
        # initiate a counter to help identify at what listing the code breaks, if it does...
        count +=1
        # print count
        
        # longitude and latitude
        g = None
        try:
            g = l.get_attribute('se:map:point')
            if g:
                lt, ln = g.split(',')
                lat.append(float(lt))
                lng.append(float(ln))
            else:
                lat.append('N/A')
                lng.append('N/A')
        except:
            lat.append('N/A')
            lng.append('N/A')
        # time.sleep(1)
        
        # address
        ad = None
        try:
            ad = l.find_element_by_class_name('details-title').text.split('\n')[0]
            if ad:
                address.append(ad)
            else:
                address.append('N/A')
        except:
            address.append('N/A')
        # time.sleep(1)
        
        # price
        p = None
        try:
            p = float(l.find_element_by_class_name('price').text.replace('$','').replace(',', ''))
            if p:
                price.append(p)
            else:
                price.append('N/A')
        except:
            price.append('N/A')
        # time.sleep(1)
        
        # number of beds
        bd_detail = None
        try:
            bd_detail = l.find_element_by_class_name('details_info').find_element_by_tag_name('span')
            if bd_detail.text.find('bed') > 0:
                no_of_beds.append(float(bd_detail.text.split(' ')[0]))
            # do we want this as a string or float? what are the regression/ml requirements?
            else:
                no_of_beds.append('N/A')
        except:
            no_of_beds.append('N/A')
        # time.sleep(1)
        
        # number of baths
        baths = None
        try:
            lstn_details = l.find_element_by_class_name('details_info').find_elements_by_tag_name('span')
            for detail in lstn_details:
                if detail.text.find('bath') > 0:
                    try:
                        baths = float(detail.text.split(' ')[0])
                    except:
                        baths = 'N/A'
        except:
            baths = 'N/A'
        no_of_baths.append(baths)
        # time.sleep(1)
        
        # square area NB: value in previous listing is being appended to next listing. FIX!
        # update: fixed.
        area = None
        try:
            l_details = l.find_element_by_class_name('details_info').find_elements_by_tag_name('span')
            for detail in l_details:
                 if detail.text.find('ft') > 0:
                    area = float(detail.text.split(' ')[0].replace(',', ''))
            if area:
                sq_area.append(area)
            else:
                sq_area.append('N/A')
        except:
            sq_area.append('N/A')
        # time.sleep(1)
        
        # listing type and neighborhood
        l_type = None
        nhood = None
        try:
            area_details = l.find_elements_by_class_name('details_info')[1].text
            l_type, nhood = area_details.split(' in ')
            if l_type:
                listing_type.append(l_type)
            else:
                listing_type.append('N/A')
            if nhood:
                n_hood.append(nhood)
            else:
                n_hood.append('N/A')
        except:
            listing_type.append('N/A')
            n_hood.append('N/A')
        # time.sleep(1)
    # if count == 14:
        # print('Moving on to the next page...')
    # streeteasy introduces a captcha when they suspect scraping. How will this be overridden?
    # fixed by using Firefox in place of Chrome
    
    return listings

In [7]:
def next_page():
    nxt = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CLASS_NAME, 'next')))
    # nxt = listns[-1].find_element_by_class_name('next')
    nxt.click()

In [8]:
@tenacity.retry()
def retry():
    driver.get(driver.current_url)
    
# OR: ret = tenacity.Retrying(retry=tenacity.retry_if_exception_type(TimeoutException))
# ret.call(driver.get(driver.currentl_url))

In [ ]:
t = time.time()
# ret = tenacity.Retrying(retry=tenacity.retry_if_exception_type(TimeoutException))
driver = webdriver.Firefox()
for x in range(0, len(urls)):
    counter = 1
    page_crash = 0
    driver.get(urls[x])
    time.sleep(2)
    listns = page_scrape(driver)
    borough.append(boroughs[x])
    last_page = int(listns[-1].find_elements_by_class_name('page')[-1].text)
    # print 'Counter:', counter
    next_page()
    for i in range(1, last_page):
        try:
            counter += 1
            b_time = time.time()
            delay = random.uniform(2.5, 5)
            listns = page_scrape(driver)
            borough.append(boroughs[x])
            next_page()
        except:
            page_crash += 1
            print 'Retrying...'
            # ret.call(driver.get(driver.current_url))
            retry()
        # print 'Counter:', counter
        time.sleep(delay)
    print time.time() - b_time
print time.time() - t

Retrying...


In [15]:
driver.close()

In [11]:
dic = {'building type':listing_type, 'borough':borough, 'latitude':lat, 'longitude':lng, 'address':address, 'beds':no_of_beds, 'baths':no_of_baths, 'area':sq_area, 'neighborhood':n_hood, 'price':price}
data = pd.DataFrame(dic)
# data.to_csv('Streeteasy_data.csv') "The data is not yet good enough to warrant saving to disk...
data.head()

,address,area,baths,beds,building type,latitude,longitude,neighborhood,price
0,100 East 53rd Street #49A,3385,3.5,3,Condo,40.7585,-73.9722,Midtown,1.325e+07
1,225 Rector Place #14D,695,1,1,Condo,40.7094,-74.016,Battery Park City,940000
2,591 Third Avenue #12D,590,1,N/A,Condo,40.7485,-73.9759,Murray Hill,970000
3,591 3rd Avenue #5E,590,1,N/A,Condo,40.7485,-73.9759,Murray Hill,935000
4,140 East 95th Street #4E,N/A,1,N/A,Co-op,40.7852,-73.9518,Carnegie Hill,399000


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12376 entries, 0 to 12375
Data columns (total 9 columns):
address          12376 non-null object
area             12376 non-null object
baths            12297 non-null object
beds             12376 non-null object
building type    12376 non-null object
latitude         12376 non-null object
longitude        12376 non-null object
neighborhood     12376 non-null object
price            12376 non-null object
dtypes: object(9)
memory usage: 870.3+ KB


In [17]:
print data.describe()

       address   area    baths     beds building type      latitude  \
count    12376  12376  12297.0  12376.0         12376  12376.000000   
unique   10444   2389     28.0     21.0             9   2066.000000   
top        N/A    N/A      1.0      2.0         Condo     40.772022   
freq        20   3717   4509.0   3803.0          6738    177.000000   

           longitude     neighborhood      price  
count   12376.000000            12376    12376.0  
unique   1724.000000              100     1971.0  
top       -73.990588  Upper West Side  2995000.0  
freq      177.000000              930      114.0  


### Handling 'N/A' values and outliers...

*Are samples with missing data discarded or replaced with the feature median? What is the norm as pertains to this situation...?*

Are statistical outliers really outliers in this use case? (Yes/**No**)?

### Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# sns.set(style='whitegrid', context='notebook')


### Model Equations
- Multivariate Linear Regression:
 - $y = w_0x_0 + w_1x_1 + ... + w_mx_m = \sum\limits_{i=0}^{\infty} w_ix_i = w^Tx$

In [ ]:
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import metrics


In [7]:
import numpy as np
help(np.c_)

Help on CClass in module numpy.lib.index_tricks object:

class CClass(AxisConcatenator)
 |  Translates slice objects to concatenation along the second axis.
 |  
 |  This is short-hand for ``np.r_['-1,2,0', index expression]``, which is
 |  useful because of its common occurrence. In particular, arrays will be
 |  stacked along their last axis after being upgraded to at least 2-D with
 |  1's post-pended to the shape (column vectors made out of 1-D arrays).
 |  
 |  For detailed documentation, see `r_`.
 |  
 |  Examples
 |  --------
 |  >>> np.c_[np.array([[1,2,3]]), 0, 0, np.array([[4,5,6]])]
 |  array([[1, 2, 3, 0, 0, 4, 5, 6]])
 |  
 |  Method resolution order:
 |      CClass
 |      AxisConcatenator
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from AxisConcatenator:
 |  
 |  __getitem__(self, key)
 |  
 |  __getslice__(self, i, j)
 |  
 |  __le

### Performance Measure

RMSE